# 动静结合

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_combine.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_combine.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/pynative_graph/combine.ipynb)

当前在业界支持动态图和静态图两种模式，动态图通过解释执行，具有动态语法亲和性，表达灵活；静态图使用jit(just in time)编译优化执行，偏静态语法，在语法上有较多限制。动态图和静态图的编译流程不一致，导致语法约束也不一致。

MindSpore针对动态图和静态图模式，首先统一API表达，在两种模式下使用相同的API；其次统一动态图和静态图的底层微分机制。

![dynamic](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/pynative_graph/images/framework1.png)

## 实现原理

MindSpore支持使用`ms_function`装饰器来修饰需要用静态图执行的对象，从而实现动静结合的目的。下面我们通过一个简单的动静结合的示例来介绍其实现原理。示例代码如下：

In [1]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_function

class Add(nn.Cell):
    """自定义类实现x自身相加"""
    def construct(self, x):
        x = x + x
        return x

class Mul(nn.Cell):
    """自定义类实现x自身相乘"""
    @ms_function  # 使用ms_function修饰，此函数以静态图方式执行
    def construct(self, x):
        x = x * x
        return x

class Test(nn.Cell):
    """自定义类实现x先Add(x)，后Mul(x)，再Add(x)"""
    def __init__(self):
        super(Test, self).__init__()
        self.add = Add()
        self.mul = Mul()

    def construct(self, x):
        x = self.add(x)
        x = self.mul(x)
        x = self.add(x)
        return x

context.set_context(mode=context.PYNATIVE_MODE)
x = Tensor(np.ones([3, 3], dtype=np.float32))
print("init x:\n", x)
net = Test()
x = net(x)
print("\nx:\n", x)

init x:
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

x:
 [[8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]


从上面的打印结果可以看出，经过Test运算后，x最终值为每个元素都是8的3\*3矩阵。该用例按照执行序，编译的方式如下图所示：

![msfunction](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/pynative_graph/images/ms_function.png)

被`ms_function`修饰的函数将会按照静态图的方式进行编译和执行。如果网络涉及到反向求导，被`ms_function`修饰的部分也将以整图的形式来生成反向图，并与前后单个算子的反向图连成一个整体后被下发执行。其中，缓存的策略与静态图的缓存策略一致，相同的函数对象在输入Shape和Type信息一致时，编译的图结构将会被缓存。

## 相互转换

在MindSpore中，我们可以通过控制模式输入参数`context.set_context`来切换执行使用动态图还是静态图。

由于在静态图下，对于Python语法有所限制，因此从动态图切换成静态图时，需要符合静态图的语法限制，才能正确使用静态图来进行执行。

> 更多静态图的语法限制可以参考[静态图语法支持](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)。

### 使用`ms_function`装饰器

MindSpore支持在动态图下使用静态编译的方式来进行混合执行，通过使用`ms_function`装饰符来修饰需要用静态图来执行的函数对象，即可实现动态图和静态图的混合执行。

#### 1. 修饰独立函数

使用`ms_function`装饰器时，可以对独立定义的函数进行修饰，使其在Graph模式下运行，示例如下：

In [2]:
import numpy as np
import mindspore.ops as ops
from mindspore import context, Tensor, ms_function

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

# 使用装饰器，指定静态图模式下执行
@ms_function
def add_func(x, y):
    return ops.add(x, y)

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

out = add_func(x, y)
print(out)

[5. 7. 9.]


在上面的示例代码中，虽然一开始设置了运行模式为动态图模式，但是由于使用了`ms_function`装饰器对函数`add_func(x, y)`进行了修饰，所以函数`add_func(x, y)`仍然是以静态图模式运行。

#### 2. 修饰Cell的成员函数

使用`ms_function`装饰器时，可以对`Cell`的成员函数进行修饰，示例代码如下：

In [3]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import context, Tensor, ms_function

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

class Add(nn.Cell):

    @ms_function # 使用装饰器，指定静态图模式下执行
    def construct(self, x, y):
        out = x + y
        return out

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

grad_ops = ops.GradOperation(get_all=True)  # 定义求导操作
net = Add()
grad_out = grad_ops(net)(x, y)

print("Infer result:\n", net(x, y))

print("Gradient result:")
print("Grad x Tensor1:\n", grad_out[0])  # 对x求导
print("Grad y Tensor2:\n", grad_out[1])  # 对y求导

Infer result:
 [5. 7. 9.]
Gradient result:
Grad x Tensor1:
 [1. 1. 1.]
Grad y Tensor2:
 [1. 1. 1.]


从上面的打印结果可以看出，x与y相加的结果为\[5, 7, 9\]， 对x求导的结果和对y求导的结果相同，都为\[1, 1, 1\]。